In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Linear Regression').getOrCreate()

In [23]:
cruise=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [24]:
cruise.count()

158

In [25]:
len(cruise.columns)

9

In [26]:
cruise.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [27]:
cruise.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [28]:
cruise.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [29]:
cruise.select('Ship_name').distinct().count()

138

In [30]:
cruise.select('Cruise_line').distinct().count()

20

In [31]:
from pyspark.ml.feature import StringIndexer

In [32]:
idx=StringIndexer(inputCol='Cruise_line',outputCol='Cruise_Cat')

In [33]:
idxd=idx.fit(cruise).transform(cruise)

In [34]:
idxd.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_Cat=16.0)

In [35]:
idxd.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_Cat']

In [36]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [37]:
assembler=VectorAssembler(
    inputCols=
    ['Age',
     'Tonnage',
     'passengers',
     'length',
     'cabins',
     'passenger_density',
     'Cruise_Cat'],
    outputCol='features')

In [38]:
output=assembler.transform(idxd)

In [39]:
output.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_Cat',
 'features']

In [41]:
output.select(['features','crew']).show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [42]:
final_output=output.select(['features','crew'])

In [43]:
final_output.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [45]:
train,test=final_output.randomSplit([.7,.3])

In [46]:
from pyspark.ml.regression import LinearRegression

In [47]:
lr=LinearRegression(labelCol='crew')

In [48]:
lrModel=lr.fit(train)

In [50]:
print(lrModel.coefficients)

[-0.00893377423926006,0.0044365791286623294,-0.11799713290128327,0.42944906576715386,0.8321054180036087,-0.00029069758922869077,0.04625598950585612]


In [53]:
predict=lrModel.evaluate(test)

In [54]:
predict.rootMeanSquaredError

0.6740872600763985

In [55]:
predict.meanAbsoluteError

0.5302159611971315

In [56]:
predict.meanSquaredError

0.4543936341973061